## What is Py(Spark)

"PySpark is the Python API for Apache Spark, a distributed computing framework for large-scale data processing."

Let's disect this sentence:

## Python API

This basically means a way to use Python to interact with Spark.

Spark is written in Scala but you can interact with Spark in several ways (e.g, R, Java and Python)

### Large-scale data processing

Spark can process Terabytes of data, and with some effort can scale up to Petabytes (https://www.databricks.com/blog/2014/10/10/spark-petabyte-sort.html)

So seriously big data processing is possible; you just need a large enough cluster/wallet.


Sidenote: It does however also mean that Spark is not very well suited for small data (up to a few million rows), it will work, but the overhead will be large. Pandas/Polars/Python would be better suited for analysis on small datasets.

### Distributed computing framework
Distributed computing is the method of making multiple computers work together to solve a common problem.

For example, we need to perform some operations on a 100 GB file. Processing this data on a single machine can take hours or maybe days based on the operation. 
However, if the same file could be broken down into 100 files of 1GB each and then processed in parallel, our total time taken would become approximately 1/100th.

A Spark cluster consists of the following components shown in the image below:

For more background information, see: https://spark.apache.org/docs/latest/cluster-overview.html

https://www.kaggle.com/datasets/neilclack/nyc-taxi-trip-data-google-public-data

In [1]:
import pandas as pd
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

sc = pyspark.SparkContext(appName="myAppName")
spark = SparkSession.builder.getOrCreate()

24/08/01 14:11:16 WARN Utils: Your hostname, Alexandrs-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 10.51.45.161 instead (on interface en0)
24/08/01 14:11:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/01 14:11:16 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# 1. Basic operations

## 1.1 Read

**Example**: reading the trips dataset from taxi_trip_data.csv file in data folder

In [2]:
sdf_trips = spark.read.csv("data/taxi_trip_data.csv", header=True, inferSchema=True)

**Task**: read the geo zones dataset from the taxi_zone_geo.csv file in the data folder

In [3]:
sdf_geo = spark.read.csv("data/taxi_zone_geo.csv", header=True, inferSchema=True)

## 1.2 Display rows

**Example**: show a sample of 5 records from sdf_trips

In [4]:
sdf_trips.show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|        2|2018-03-29 13:37:13|2018-03-29 14:17:01|              1|        18.15|        3|                 N|           1|       70.0|  0.0|    0.0|     16.16|        10.5|          0.3|       96.96|               161|                  1|
|        2|2018-03-29 13:37:18|2018-03-2

**Task**: show a sample of 3 records from sdf_geo

In [5]:
sdf_geo.show(3)

+-------+--------------------+-------+--------------------+
|zone_id|           zone_name|borough|           zone_geom|
+-------+--------------------+-------+--------------------+
|      1|      Newark Airport|    EWR|POLYGON((-74.1856...|
|      3|Allerton/Pelham G...|  Bronx|POLYGON((-73.8485...|
|     18|        Bedford Park|  Bronx|POLYGON((-73.8844...|
+-------+--------------------+-------+--------------------+
only showing top 3 rows



## 1.3 Aggregate

**Example**: show the top 3 most used payment types

In [6]:
(
    sdf_trips
    .groupBy('payment_type')
    .count()
    .sort(F.desc('count'))
).show(3)

+------------+-------+
|payment_type|  count|
+------------+-------+
|           1|8255092|
|           2|1623133|
|           3|  95464|
+------------+-------+
only showing top 3 rows



**Task**: show the top 3 borough names with the most zone ids

Extra credit for displaying the percentage of total zone ids

In [7]:
geo_row_count = sdf_geo.count()

(
    sdf_geo
    .groupBy('borough')
    .count()
    .sort(F.desc('count'))
    .withColumn('percentage', F.format_string('%2.2f%%', 100*F.col('count')/geo_row_count))
    .drop('all_counts')
).show(5)

+-------------+-----+----------+
|      borough|count|percentage|
+-------------+-----+----------+
|       Queens|   69|    26.24%|
|    Manhattan|   69|    26.24%|
|     Brooklyn|   61|    23.19%|
|        Bronx|   43|    16.35%|
|Staten Island|   20|     7.60%|
+-------------+-----+----------+
only showing top 5 rows



## 1.4 Filter

**Example**: count of the rides with more than 3 passengers in the year May of 2018

In [8]:
n_full_car_rides = (
    sdf_trips
    .filter(F.date_format(F.col('pickup_datetime'), "MMyyyy") == "052018")
    .filter(F.col('passenger_count') > 4)
).count()

print(f"{n_full_car_rides:,}")

24/08/01 14:11:29 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


64,864


**Task**: find the cost of the most expensive ride that ended between 5pm and 6pm made paid with the payment type 1

In [9]:
cost_evening_ride = (
    sdf_trips
    # .withColumn("hours", F.date_format(F.col('pickup_datetime'), "HH"))
    .filter(F.date_format(F.col('dropoff_datetime'), "HH") == "17")
    .filter(F.col('payment_type') == 1)
    .agg(F.max("fare_amount"))
).collect()[0][0]

print(f"${cost_evening_ride:,.2f}")

$699.20


## 1.5 Deduplicate

**Example**: keep only one trip for each unique combination of passenger_count and rate_code

In [10]:
(
    sdf_trips
    .dropDuplicates(subset=["passenger_count", "rate_code"])
).show(5)

24/08/01 14:11:33 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|        1|2018-11-01 15:36:34|2018-11-01 16:08:56|              0|          8.0|        1|                 N|           1|       27.5|  0.0|    0.5|      5.65|         0.0|          0.3|       33.95|               138|                255|
|        1|2018-10-31 14:49:28|2018-10-3

**Task**: keep only one zone id for each zone name

In [11]:
(
    sdf_geo
    .dropDuplicates(subset=["zone_name"])
    .groupBy('zone_name')
    .count()
    .sort(F.desc('count'))
).show(5)

+--------------------+-----+
|           zone_name|count|
+--------------------+-----+
|           Homecrest|    1|
|Governor's Island...|    1|
|              Corona|    1|
|    Bensonhurst West|    1|
|         Westerleigh|    1|
+--------------------+-----+
only showing top 5 rows



## 1.6 Change conditionally

**Example**: reduce the `tolls_amount` by half for the trips that started and ended in the different zones

In [12]:

(
    sdf_trips
    .withColumn(
        "new_toll",
        F.when(F.col("pickup_location_id") != F.col("dropoff_location_id"), F.col("tolls_amount") * 0.5)
        .otherwise(F.col("tolls_amount"))
    )
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+--------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|new_toll|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+--------+
|        2|2018-03-29 13:37:13|2018-03-29 14:17:01|              1|        18.15|        3|                 N|           1|       70.0|  0.0|    0.0|     16.16|        10.5|          0.3|       96.96|               161|                  1|    5.25|
|   

**Task**: set the negative fare_amount to 0 for the negative fares and cap the positive ones by $10

In [13]:
(
    sdf_trips
    .withColumn(
        "fare_amount",
        F.when(F.col("fare_amount") < 0, F.lit(0))
        .when(F.col("fare_amount") > 10, F.lit(10))
        .otherwise(F.col("fare_amount"))
    )
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|        2|2018-03-29 13:37:13|2018-03-29 14:17:01|              1|        18.15|        3|                 N|           1|       10.0|  0.0|    0.0|     16.16|        10.5|          0.3|       96.96|               161|                  1|
|        2|2018-03-29 13:37:18|2018-03-2

## 1.7 Join

**Example**: how many single passengers were picked up in Bronx borough

In [14]:
n_single_bronx_rides = (
    sdf_trips
    .filter(F.col("passenger_count") == 1)
    .join(
        sdf_geo
        .filter(F.col("borough") == "Bronx"),
        sdf_trips.pickup_location_id == sdf_geo.zone_id,
        how="inner",
    )
).count()

print(f"{n_single_bronx_rides:,} single Bronx rides")

20,057 single Bronx rides


**Task**: What is the longest ride to JFK Airport zone for less than $20?

In [15]:
longest_cheap_trip = (
    sdf_trips
    .filter(F.col("passenger_count") == 1)
    .join(
        sdf_geo
        .filter(F.col("zone_name") == "JFK Airport"),
        sdf_trips.dropoff_location_id == sdf_geo.zone_id,
        how="inner",
    )
    .filter(F.col("fare_amount") < 20)
    .sort(F.desc("trip_distance"))
    .limit(1)
).collect()[0]["trip_distance"]

print(f"Price for the longest cheap trip: ${longest_cheap_trip:,.2f}")

Price for the longest cheap trip: $132.90


# 2. User defined functions
When you think you ran out of Pyspark native options

In [16]:
import pyspark.sql.types as T

**Example**: round up the tip amount

In [17]:
from math import ceil

def spark_ceil(x):
    return ceil(x)

spark_ceil_udf = F.udf(spark_ceil, T.IntegerType())

In [18]:
(
    sdf_trips
    .withColumn("tip_amount_rounded_up", spark_ceil_udf(F.col("tip_amount")))
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+---------------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|tip_amount_rounded_up|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+---------------------+
|        2|2018-03-29 13:37:13|2018-03-29 14:17:01|              1|        18.15|        3|                 N|           1|       70.0|  0.0|    0.0|     16.16|        10.5|          0.3|       96.96|              

**Task**: calculate cosine of the tolls amount

In [19]:
from math import cos

def spark_cos(x):
    return cos(x)

spark_cos_udf = F.udf(spark_cos, T.FloatType())

In [20]:
(
    sdf_trips
    .withColumn("cos_tolls_amount", spark_cos_udf(F.col("tolls_amount")))
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|cos_tolls_amount|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------------+
|        2|2018-03-29 13:37:13|2018-03-29 14:17:01|              1|        18.15|        3|                 N|           1|       70.0|  0.0|    0.0|     16.16|        10.5|          0.3|       96.96|               161|          

# 3. Transformations within slices/windows

In [21]:
from pyspark.sql import Window

**Example**: keep the 3 latest rides from each pickup zone

In [22]:
(
    sdf_trips
    .withColumn(
        "row_number",
        F.row_number()
        .over(
            Window
            .partitionBy("pickup_location_id")
            .orderBy(F.desc("pickup_datetime"))
        )
    )
    .filter(F.col("row_number") < 4)
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|row_number|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------+
|        1|2018-12-31 15:04:52|2018-12-31 15:05:25|              1|         13.2|        5|                 N|           1|      162.5|  0.0|    0.0|      28.0|         0.0|          0.3|       190.8|                 1|                  1|        

**Task**: calculate a cumulative sum of mta_tax starting from the earliest for each payment type

Extra credit: how many trips did it take to accumulate $5 in mta_tax for each payment type?

In [23]:
TO_ACCUMULATE = 5

(
    sdf_trips
    .withColumn(
        "cumulative_mta_tax",
        F.sum("mta_tax")
        .over(
            Window
            .partitionBy("payment_type")
            .orderBy(F.col("pickup_datetime"))
            .rowsBetween(Window.unboundedPreceding, Window.currentRow)
        )
    )
    .filter(F.col("cumulative_mta_tax") <= TO_ACCUMULATE)
    .groupBy("payment_type")
    .agg(
        F.count("*").alias("trips_count"),
        F.max("cumulative_mta_tax").alias("max_cum")
    )
    .select(
        "payment_type",
        F.when(F.col("max_cum") >= TO_ACCUMULATE, F.col("trips_count"))
        .otherwise(F.lit(None))
        .alias("trips_count")
    )
).show(50)

24/08/01 14:12:02 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+------------+-----------+
|payment_type|trips_count|
+------------+-----------+
|           1|         10|
|           3|         14|
|           4|         54|
|           5|       NULL|
|           2|         10|
+------------+-----------+



# 4. Structures and Arrays

## 4.1 Structures

**Example**: put pickup and dropoff locations to the zone_ids structure

In [24]:
(
    sdf_trips
    .withColumn(
        "zone_ids",
        F.struct('pickup_location_id','dropoff_location_id')
    )
).printSchema()

root
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- imp_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- zone_ids: struct (nullable = false)
 |    |-- pickup_location_id: integer (nullable = true)
 |    |-- dropoff_location_id: integer (nullable = true)



**Task 1**: create a column that would hold a structure called payments that would include the total and substructure with all the contributions to this payment

**Task 2**: create a dataframe `sdf_structured` with the column above while the contributing columns are dropped

In [25]:
sdf_structured = (
    sdf_trips
    .withColumn(
        "contributions",
        F.struct(*sdf_trips.columns[8:12])
    )
    .withColumn(
        'payments',
        F.struct("contributions", "total_amount")
    )
    .drop(*sdf_trips.columns[8:13])
    .drop("contributions")
)

sdf_structured.printSchema()

root
 |-- vendor_id: integer (nullable = true)
 |-- pickup_datetime: timestamp (nullable = true)
 |-- dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- rate_code: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- imp_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- pickup_location_id: integer (nullable = true)
 |-- dropoff_location_id: integer (nullable = true)
 |-- payments: struct (nullable = false)
 |    |-- contributions: struct (nullable = false)
 |    |    |-- fare_amount: double (nullable = true)
 |    |    |-- extra: double (nullable = true)
 |    |    |-- mta_tax: double (nullable = true)
 |    |    |-- tip_amount: double (nullable = true)
 |    |-- total_amount: double (nullable = true)



## 4.2 Arrays

**Example**: collect an array of trip distances for each `passenger_count`, `rate_code`, and `payment_type` combination

In [26]:
(
    sdf_trips
    .groupBy(
        "passenger_count",
        "rate_code",
        "payment_type",
    )
    .agg(
        F.collect_list('trip_distance').alias('distances')
    )
).show(5)

+---------------+---------+------------+--------------------+
|passenger_count|rate_code|payment_type|           distances|
+---------------+---------+------------+--------------------+
|              1|        5|           3|[0.0, 8.0, 0.0, 0...|
|              3|        2|           3|[0.33, 0.06, 0.0,...|
|              5|        4|           2|[8.38, 21.58, 16....|
|              3|        1|           4|[9.8, 6.1, 0.21, ...|
|              3|        3|           1|[16.6, 17.6, 17.6...|
+---------------+---------+------------+--------------------+
only showing top 5 rows



**Task**: create a column that holds the 5 latest payments structures from `sdf_structured` for each pickup location and store the resulting data frame as `sdf_structured_arrayed`

**Extra Credit**: extract the 3rd tip amount from the array for each location

In [27]:
sdf_structured_arrayed = (
    sdf_structured
    .withColumn(
        "row_number",
        F.row_number()
        .over(
            Window
            .partitionBy("pickup_location_id")
            .orderBy(F.desc("pickup_datetime"))
        )
    )
    .filter(F.col("row_number") < 6)
    .groupBy("pickup_location_id")
    .agg(
        F.collect_list('payments').alias('payments')
    )
    .withColumnRenamed("pickup_location_id", "zone_id")
    .join(
        sdf_geo,
        on="zone_id",
        how="left",
    )
)
sdf_structured_arrayed.printSchema()

(
    sdf_structured_arrayed
    .select(
        "zone_name",
        "payments",
        (F.col("payments.contributions.tip_amount")[2]).alias('the_trird_tip')
    )
).show(5)

root
 |-- zone_id: integer (nullable = true)
 |-- payments: array (nullable = false)
 |    |-- element: struct (containsNull = false)
 |    |    |-- contributions: struct (nullable = false)
 |    |    |    |-- fare_amount: double (nullable = true)
 |    |    |    |-- extra: double (nullable = true)
 |    |    |    |-- mta_tax: double (nullable = true)
 |    |    |    |-- tip_amount: double (nullable = true)
 |    |    |-- total_amount: double (nullable = true)
 |-- zone_name: string (nullable = true)
 |-- borough: string (nullable = true)
 |-- zone_geom: string (nullable = true)



+--------------------+--------------------+-------------+
|           zone_name|            payments|the_trird_tip|
+--------------------+--------------------+-------------+
|      Newark Airport|[{{162.5, 0.0, 0....|         15.0|
|         Jamaica Bay|[{{52.0, 0.0, 0.5...|         14.6|
|Allerton/Pelham G...|[{{45.0, 0.0, 0.0...|          0.0|
|       Alphabet City|[{{37.0, 0.5, 0.5...|          0.0|
|       Arden Heights|[{{78.5, 0.0, 0.5...|          0.0|
+--------------------+--------------------+-------------+
only showing top 5 rows



## 4.3 Extract data

**Example**: Extract total amount from `sdf_structured` into individual rows

In [28]:
(
    sdf_structured_arrayed
    .withColumn('payments', F.explode('payments'))
    .select("*", "payments.total_amount")
).show(5)

+-------+--------------------+--------------+-------+--------------------+------------+
|zone_id|            payments|     zone_name|borough|           zone_geom|total_amount|
+-------+--------------------+--------------+-------+--------------------+------------+
|      1|{{162.5, 0.0, 0.0...|Newark Airport|    EWR|POLYGON((-74.1856...|       190.8|
|      1|{{95.0, 0.0, 0.0,...|Newark Airport|    EWR|POLYGON((-74.1856...|       100.0|
|      1|{{120.0, 0.0, 0.0...|Newark Airport|    EWR|POLYGON((-74.1856...|       135.3|
|      1|{{80.0, 0.0, 0.0,...|Newark Airport|    EWR|POLYGON((-74.1856...|      114.35|
|      1|{{100.0, 0.0, 0.0...|Newark Airport|    EWR|POLYGON((-74.1856...|      132.96|
+-------+--------------------+--------------+-------+--------------------+------------+
only showing top 5 rows



**Task**: Extract contributions from `sdf_structured` into individual rows

In [29]:
(
    sdf_structured_arrayed
    .withColumn('payments', F.explode('payments'))
    .select("*", "payments.contributions.*")
).show(5)

+-------+--------------------+--------------+-------+--------------------+-----------+-----+-------+----------+
|zone_id|            payments|     zone_name|borough|           zone_geom|fare_amount|extra|mta_tax|tip_amount|
+-------+--------------------+--------------+-------+--------------------+-----------+-----+-------+----------+
|      1|{{162.5, 0.0, 0.0...|Newark Airport|    EWR|POLYGON((-74.1856...|      162.5|  0.0|    0.0|      28.0|
|      1|{{95.0, 0.0, 0.0,...|Newark Airport|    EWR|POLYGON((-74.1856...|       95.0|  0.0|    0.0|       4.7|
|      1|{{120.0, 0.0, 0.0...|Newark Airport|    EWR|POLYGON((-74.1856...|      120.0|  0.0|    0.0|      15.0|
|      1|{{80.0, 0.0, 0.0,...|Newark Airport|    EWR|POLYGON((-74.1856...|       80.0|  0.0|    0.0|     19.05|
|      1|{{100.0, 0.0, 0.0...|Newark Airport|    EWR|POLYGON((-74.1856...|      100.0|  0.0|    0.0|     22.16|
+-------+--------------------+--------------+-------+--------------------+-----------+-----+-------+----

# 5. Time

## 5.1 Format time

**Example**: keep only the rides that went over at least one night and change pickup and dropoff times to the format like `April 27, 1967`

Make sure to exclude the ones traveling back in time (dropped off before picked up)

In [30]:
(
    sdf_trips
    .withColumn(
        "date_diff",
        F.datediff('dropoff_datetime', 'pickup_datetime')
    )
    .filter(F.col("date_diff") > 0)
    .withColumn('pickup_date', F.date_format(F.col("pickup_datetime"), 'MMMM d, yyy'))
    .withColumn('dropoff_date', F.date_format(F.col("dropoff_datetime"), 'MMMM d, yyy'))
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+---------+--------------+--------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|date_diff|   pickup_date|  dropoff_date|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+---------+--------------+--------------+
|        1|2018-05-18 23:46:46|2018-05-19 00:11:22|              1|          7.5|        1|                 N|           1|       24.5|  0.5|    0.5|      6.45|

**Task**: keep only the weekend rides (started on either Saturday or Sunday) and specify which day it was in the corresponding column

In [31]:
(
    sdf_trips
    .withColumn(
        "day_of_week",
        F.dayofweek('pickup_datetime')
    )
    .filter(F.col("day_of_week").isin([1, 7]))
    .withColumn(
        "day_of_week",
        F.when(F.col("day_of_week") == 1, F.lit("Sunday"))
        .otherwise(F.lit("Saturday"))
    )
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+-----------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|day_of_week|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+-----------+
|        1|2018-05-19 00:20:08|2018-05-19 00:56:25|              1|          6.4|        1|                 N|           1|       26.5|  0.5|    0.5|       4.0|         0.0|          0.3|        31.8|               249|                226|   Sa

## 5.2 Unix time

**Example**: keep only the rides that ended on the day corresponding to the following unix timestamp `1521552311`

In [32]:
(
    sdf_trips
    .filter(F.to_date("dropoff_datetime") == F.to_date(F.from_unixtime(F.lit("1521552311"))))
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|        1|2018-03-20 20:31:34|2018-03-20 21:08:54|              1|          6.7|        1|                 N|           1|       27.0|  0.5|    0.5|       4.0|         0.0|          0.3|        32.3|               161|                181|
|        2|2018-03-20 20:37:49|2018-03-2

**Task**: find the date corresponding twice the unix pickup date for each ride

In [33]:
(
    sdf_trips
    .withColumn(
        "ts_doubled",
        2*F.unix_timestamp('pickup_datetime')
    )
    .withColumn(
        "doubled_date",
        F.to_date(F.from_unixtime(F.col("ts_doubled")))
    )
    .sample(0.01)
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------+------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|ts_doubled|doubled_date|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------+------------+
|        2|2018-02-01 19:45:04|2018-02-01 19:45:06|              2|          0.0|        5|                 N|           1|      186.5|  0.0|    0.5|     37.46|         0.0|          0.3|      224.76|        

# 6. Drying the code

## 6.1 Windows

**Example**: rewrite the example in part 3 with window defined in a separate variable

In [34]:
window_pickup = Window.partitionBy("pickup_location_id").orderBy(F.desc("pickup_datetime"))

(
    sdf_trips
    .withColumn("row_number", F.row_number().over(window_pickup))
    .filter(F.col("row_number") < 4)
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|row_number|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+----------+
|        1|2018-12-31 15:04:52|2018-12-31 15:05:25|              1|         13.2|        5|                 N|           1|      162.5|  0.0|    0.0|      28.0|         0.0|          0.3|       190.8|                 1|                  1|        

**Task**: rewrite the task from part 3 with window defined in a separate variable

**Extra credit**: rewrite the filtering condition as a separate variable

In [35]:
TO_ACCUMULATE = 5
window_payment_type = Window.partitionBy("payment_type").orderBy(F.col("pickup_datetime")).rowsBetween(Window.unboundedPreceding, Window.currentRow)
limit_cum_mta = F.col("cumulative_mta_tax") <= TO_ACCUMULATE

(
    sdf_trips
    .withColumn(
        "cumulative_mta_tax",
        F.sum("mta_tax")
        .over(window_payment_type)
    )
    .filter(limit_cum_mta)
    .groupBy("payment_type")
    .agg(
        F.count("*").alias("trips_count"),
        F.max("cumulative_mta_tax").alias("max_cum")
    )
    .select(
        "payment_type",
        F.when(F.col("max_cum") >= TO_ACCUMULATE, F.col("trips_count"))
        .otherwise(F.lit(None))
        .alias("trips_count")
    )
).show(50)

24/08/01 14:12:40 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+------------+-----------+
|payment_type|trips_count|
+------------+-----------+
|           1|         10|
|           3|         14|
|           4|         54|
|           5|       NULL|
|           2|         10|
+------------+-----------+



## 6.2 Parameterization

**Example**: multiply each of the monetary columns by the number of symbols in the column name and add this value to the column name

In [36]:
(
    sdf_trips
    .select(
        *sdf_trips.columns[:8],
        *[(F.col(c)*len(c)).alias(f"{c} * {len(c)}") for c in sdf_trips.columns[8:-2]],
        *sdf_trips.columns[8:-2],
        *sdf_trips.columns[-2:],
    )
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+----------------+---------+-----------+---------------+-----------------+------------------+-----------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount * 11|extra * 5|mta_tax * 7|tip_amount * 10|tolls_amount * 12|imp_surcharge * 13|total_amount * 12|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+----------------+---------+-----------+---------------+-----------------+------------------+-----------------+-----------+-----+-------+----------+------------+-------------+---

**Task**: rewrite the example from 5.1 by creating a function that takes the date time column name and returns the formatted output

**Extra credit**: make the function work in a loop by providing a list of columns to transform

In [37]:
def transform_date(col):
    return F.date_format(F.col(col), 'MMMM d, yyy')

(
    sdf_trips
    .withColumn(
        "date_diff",
        F.datediff('dropoff_datetime', 'pickup_datetime')
    )
    .filter(F.col("date_diff") > 0)
    .select(
        "*",
        *[transform_date(col).alias(col[:-4])
          for col in sdf_trips.columns
          if "datetime" in col]
    )
).show(5)

+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+---------+--------------+--------------+
|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|trip_distance|rate_code|store_and_fwd_flag|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|imp_surcharge|total_amount|pickup_location_id|dropoff_location_id|date_diff|   pickup_date|  dropoff_date|
+---------+-------------------+-------------------+---------------+-------------+---------+------------------+------------+-----------+-----+-------+----------+------------+-------------+------------+------------------+-------------------+---------+--------------+--------------+
|        1|2018-05-18 23:46:46|2018-05-19 00:11:22|              1|          7.5|        1|                 N|           1|       24.5|  0.5|    0.5|      6.45|

# 7. Prototyping

## 7.1 Sampling

**Example**: calculate the number of combinations of rides such that the drop off location for one is the pick up location for another

**Hint**: stop the execution if spent more than 5 minutes waiting and proceed to the following task

In [38]:
sdf_pairs = (
    sdf_trips
    .select(
        F.col("pickup_location_id").alias("pickup_location_id_a"),
        F.col("dropoff_location_id").alias("dropoff_location_id_a"),
        F.col("dropoff_location_id").alias("to_join"),
    )
    .join(
        sdf_trips
        .select(
            F.col("pickup_location_id").alias("pickup_location_id_b"),
            F.col("dropoff_location_id").alias("dropoff_location_id_b"),
            F.col("pickup_location_id").alias("to_join"),
        ),
        on="to_join",
        how='inner',
    )
)

In [39]:
# counting the combinations
# print(f"{sdf_sampled_pairs.count()=:,}")

**Task**: prototype the code in example for the 0.1% sample of the rides

In [40]:
sdf_trips_sample = sdf_trips.sample(0.001)

sdf_sampled_pairs = (
    sdf_trips_sample
    .select(
        F.col("pickup_location_id").alias("pickup_location_id_a"),
        F.col("dropoff_location_id").alias("dropoff_location_id_a"),
        F.col("dropoff_location_id").alias("to_join"),
    )
    .join(
        sdf_trips_sample
        .select(
            F.col("pickup_location_id").alias("pickup_location_id_b"),
            F.col("dropoff_location_id").alias("dropoff_location_id_b"),
            F.col("pickup_location_id").alias("to_join"),
        ),
        on="to_join",
        how='inner',
    )
)

print(f"{sdf_sampled_pairs.count()=:,}")

sdf_sampled_pairs.count()=2,992,180


## 7.2 Saving intermediate results

**Example**: For the task in 7.1: save the first 0.1% of rides before joining it to the whole dataset

In [41]:
sdf_trips_sample = sdf_trips.sample(0.001)

# (
#     sdf_trips_sample
#     .write.mode("overwrite")
#     .option("overwriteSchema", "True")
#     .format("parquet")
#     .saveAsTable("sdf_trips_sample")
# )

# To be continued


**Task**: process at least 10 batches of the 0.1% samples to get the corresponding scores

In [42]:
# TODO

## 7.3 Duck typing

**Example**: make `.prc()` applied to a dataframe print row count of that dataframe and test it on `sdf_trips` dataset 

In [43]:
from pyspark.sql.dataframe import DataFrame

def _prc(self):
    print(f"{self.count():,}")

DataFrame.prc = _prc

sdf_trips.prc()

10,000,000


**Task**: make `.pvc(col)` applied to a dataframe return another dataframe with row count for each unique entry in the column `col` and test it on `sdf_geo` dataset column `borough`

**Extra credit** the dataframe should also contain a column for the percentage of each unique entry

In [44]:
def _pvc(self, col):
    return (
        self
        .withColumn('total_count', F.count("*").over(Window.partitionBy(F.lit(1))))
        .groupBy(col)
        .agg(
            F.count("*").alias("count"),
            F.format_string('%.1f%%', 100*F.count("*")/F.max('total_count')).alias("percentage"),
        )
        .sort(F.desc("count"))
    )

DataFrame.pvc = _pvc

sdf_geo.pvc('borough').show()


+-------------+-----+----------+
|      borough|count|percentage|
+-------------+-----+----------+
|       Queens|   69|     26.2%|
|    Manhattan|   69|     26.2%|
|     Brooklyn|   61|     23.2%|
|        Bronx|   43|     16.3%|
|Staten Island|   20|      7.6%|
|          EWR|    1|      0.4%|
+-------------+-----+----------+

